In [1]:
import paddle
from paddle.nn import Embedding,TransformerEncoderLayer,TransformerEncoder,TransformerDecoderLayer,TransformerDecoder,Linear,Dropout
import paddle.nn.functional as F
import re
from paddle import tensor
import numpy as np

In [2]:
lines=open("data/data78721/cmn.txt",'r',encoding='utf-8').readlines()

# 一、指定文本长度，将数据处理成【【英词】，【中字】】存到datas里

In [3]:
num_data=len(lines)
num_data

24360

In [4]:
datas=[]
num_enlen={}
num_cnlen={}
for line in lines:
    line=line.strip().split('\t')
    en=line[0].lower()
    ch=en[-1]
    en=en[:-1].split(' ')
    en.append(ch)
    cn=line[1]
    cn=list(cn)
    # print(en,cn)
    datas.append([en,cn])
    if len(en) not in num_enlen:
        num_enlen[len(en)]=0
    num_enlen[len(en)]+=1

    if len(cn) not in num_cnlen:
        num_cnlen[len(cn)]=0
    num_cnlen[len(cn)]+=1

In [5]:
cnt=0
for klen,num in num_enlen.items():
    cnt+=num
    print("len:{},occupy:{}".format(klen,cnt/num_data))


len:2,occupy:0.001026272577996716
len:3,occupy:0.02167487684729064
len:4,occupy:0.08078817733990148
len:5,occupy:0.2154761904761905
len:6,occupy:0.39392446633825945
len:7,occupy:0.6069376026272578
len:8,occupy:0.8046387520525452
len:9,occupy:0.8790229885057471
len:10,occupy:0.9263957307060755
len:11,occupy:0.9539819376026273
len:12,occupy:0.9741789819376027
len:13,occupy:0.9850574712643678
len:14,occupy:0.9916666666666667
len:15,occupy:0.9951149425287357
len:17,occupy:0.9964285714285714
len:16,occupy:0.9984400656814449
len:18,occupy:0.9990147783251232
len:19,occupy:0.9994663382594418
len:20,occupy:0.9996305418719211
len:22,occupy:0.9997536945812808
len:23,occupy:0.9997947454844006
len:21,occupy:0.9998768472906404
len:26,occupy:0.9999178981937603
len:27,occupy:0.9999589490968801
len:33,occupy:1.0


In [6]:
cnt=0
for klen,num in num_cnlen.items():
    cnt+=num
    print("len:{},occupy:{}".format(klen,cnt/num_data))

len:2,occupy:0.00036945812807881776
len:3,occupy:0.004638752052545156
len:5,occupy:0.045361247947454844
len:4,occupy:0.05993431855500821
len:7,occupy:0.17311165845648604
len:6,occupy:0.25135467980295567
len:9,occupy:0.3886288998357964
len:8,occupy:0.5231937602627258
len:11,occupy:0.6228653530377668
len:10,occupy:0.7520114942528736
len:14,occupy:0.7869868637110017
len:13,occupy:0.8429392446633825
len:12,occupy:0.9151888341543514
len:16,occupy:0.9329228243021347
len:15,occupy:0.9569376026272578
len:20,occupy:0.9614942528735633
len:18,occupy:0.9704022988505747
len:17,occupy:0.9822660098522168
len:19,occupy:0.988752052545156
len:23,occupy:0.9902298850574712
len:21,occupy:0.9928571428571429
len:22,occupy:0.9952380952380953
len:25,occupy:0.9963054187192119
len:28,occupy:0.9967159277504105
len:26,occupy:0.9976190476190476
len:24,occupy:0.9984400656814449
len:34,occupy:0.9985221674876847
len:27,occupy:0.9988505747126437
len:29,occupy:0.999055829228243
len:36,occupy:0.9990968801313629
len:39,oc

In [7]:
en_len=10
cn_len=10

# 二、构建词表，加入\<pad>,\<bos>,\<eos>

In [8]:
en_vocab={}
cn_vocab={}
en_vocab["<pad>"]=0
en_vocab["<bos>"]=1
en_vocab["<eos>"]=2
cn_vocab["<pad>"]=0
cn_vocab["<bos>"]=1
cn_vocab["<eos>"]=2

In [9]:
en_idx,cn_idx=3,3

In [10]:
for en,cn in datas:
    for word in en:
        if word not in en_vocab:
            en_vocab[word]=en_idx
            en_idx+=1
    for word in cn:
        if word not in cn_vocab:
            cn_vocab[word]=cn_idx
            cn_idx+=1

In [11]:
print(len(list(en_vocab)))
len(list(cn_vocab))

7677


3533

# 三、改格式
### ①en编码输入格式:en+\<eos>+\<pad>...       len=en_len+1
### ②cn解码输入格式:\<bos>+cn+\<eos>+\<pad>... len=cn_len+2
### ③cn解码输出格式:cn+\<eos>+\<pad>...       len=cn_len+2

In [12]:
co_in=[]
de_in=[]
de_out=[]

In [13]:
for en,cn in datas:
    en=[en_vocab[w] for w in en][:en_len]
    cn=[cn_vocab[w] for w in cn][:cn_len]
    co_in.append(np.array(en+[en_vocab["<eos>"]]+[en_vocab["<pad>"]]*(en_len-len(en))))
    de_in.append(np.array([cn_vocab["<bos>"]]+cn+[cn_vocab["<eos>"]]+[cn_vocab["<pad>"]]*(cn_len-len(cn))))
    de_out.append(np.array(cn+[cn_vocab["<eos>"]]+[cn_vocab["<pad>"]]*(cn_len-len(cn)+1)))
co_in=np.array(co_in)
de_in=np.array(de_in)
de_out=np.array(de_out)
co_in.shape,de_in.shape,de_out.shape

((24360, 11), (24360, 12), (24360, 12))

# 四、模型配置

In [14]:
embedding_size=512
hidden_size=512
num_encoder_lstm_layers=1
en_vocab_size=len(list(en_vocab))
cn_vocab_size=len(list(cn_vocab))
epochs=20
batch_size=32


In [15]:
class Encoder(paddle.nn.Layer):
    def __init__(self,en_vocab_size,embedding_size,num_layers=6,head_number=8,middle_units=512):
        super(Encoder,self).__init__()
        self.emb=Embedding(en_vocab_size,embedding_size)
        encoder_layer=TransformerEncoderLayer(embedding_size,head_number,middle_units)
        self.encoder=TransformerEncoder(encoder_layer,num_layers)
    
    def forward(self,x):
        x=self.emb(x)
        en_out=self.encoder(x)
        return en_out

In [16]:
class Decoder(paddle.nn.Layer):
    def __init__(self,cn_vocab_size,embedding_size,num_layers=6,head_num=8,middle_units=512):
        super(Decoder,self).__init__()
        self.emb=Embedding(cn_vocab_size,embedding_size)
        decoder_layer=TransformerDecoderLayer(embedding_size,head_num,middle_units)
        self.decoder=TransformerDecoder(decoder_layer,num_layers)
        self.outlinear=paddle.nn.Linear(embedding_size,cn_vocab_size)

    def forward(self,x,encoder_output):
        x=self.emb(x)
        de_out=self.decoder(x,encoder_output)
        output=self.outlinear(de_out)
        output=paddle.squeeze(output)
        return output

# 五、模型训练

In [17]:
encoder=Encoder(en_vocab_size,embedding_size)
decoder=Decoder(cn_vocab_size,embedding_size)
opt=paddle.optimizer.Adam(learning_rate=0.00001,parameters=encoder.parameters()+decoder.parameters())

for epoch in range(epochs):
    print("epoch:{}".format(epoch))

    perm=np.random.permutation(len(co_in))
    co_in_shuffle=co_in[perm]
    de_in_shuffle=de_in[perm]
    de_out_shuffle=de_out[perm]
    # print("checkpont1")
    for i in range(co_in.shape[0]//batch_size):

        x=co_in_shuffle[i*batch_size:(i+1)*batch_size]
        x=paddle.to_tensor(x)
        x_repr=encoder(x)

        # print("checkpont2")

        din=de_in_shuffle[i*batch_size:(i+1)*batch_size]
        label=de_out_shuffle[i*batch_size:(i+1)*batch_size]

        # print("checkpont3")

        loss=paddle.zeros([1])

        for cur in range(cn_len+2):
            cn_word=paddle.to_tensor(din[:,cur:cur+1])
            cn_label=paddle.to_tensor(label[:,cur:cur+1])

            logits=decoder(cn_word,x_repr)
            loss+=F.cross_entropy(logits,cn_label)
        
        loss/=(cn_len+2)
        if(i%50)==0:
            print("i={},loss:{}".format(i,loss.numpy()))
        
        loss.backward()
        opt.step()
        opt.clear_grad()

W0725 11:19:35.670452   314 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W0725 11:19:35.674077   314 gpu_resources.cc:91] device: 0, cuDNN Version: 7.6.


epoch:0
i=0,loss:[8.267305]
i=50,loss:[6.1329074]
i=100,loss:[5.5076885]
i=150,loss:[5.3175797]
i=200,loss:[5.1586514]
i=250,loss:[5.230085]
i=300,loss:[4.6906333]
i=350,loss:[4.975925]
i=400,loss:[4.800014]
i=450,loss:[4.6718807]
i=500,loss:[4.4407625]
i=550,loss:[4.4118605]
i=600,loss:[4.2642937]
i=650,loss:[4.463262]
i=700,loss:[4.473484]
i=750,loss:[4.163823]
epoch:1
i=0,loss:[4.0492516]
i=50,loss:[4.204397]
i=100,loss:[4.38529]
i=150,loss:[3.8882565]
i=200,loss:[4.5078626]
i=250,loss:[4.1971846]
i=300,loss:[3.9473147]
i=350,loss:[4.0964384]
i=400,loss:[4.2042265]
i=450,loss:[4.2426767]
i=500,loss:[4.007829]
i=550,loss:[3.5777607]
i=600,loss:[4.2795687]
i=650,loss:[3.9516885]
i=700,loss:[4.316311]
i=750,loss:[3.5594516]
epoch:2
i=0,loss:[3.6666849]
i=50,loss:[3.6980581]
i=100,loss:[3.9007401]
i=150,loss:[4.003602]
i=200,loss:[3.9595745]
i=250,loss:[4.4667583]
i=300,loss:[3.4938526]
i=350,loss:[3.5525336]
i=400,loss:[3.880428]
i=450,loss:[4.1041775]
i=500,loss:[3.8872812]
i=550,loss

# 六、模型预测

In [26]:
encoder.eval()
decoder.eval()

nums=10
indices=np.random.choice(len(co_in),nums,replace=False)

x_data=co_in[indices]
sent=paddle.to_tensor(x_data)
en_repr=encoder(sent)

words=[[cn_vocab["<bos>"]]]*nums
words=np.array(words)
words=paddle.to_tensor(words)
print(words)
de_sent=[]
for i in range(cn_len+2):
    logits=decoder(words,en_repr)
    print(logits)
    words=paddle.argmax(logits,axis=1)
    print(words)
    de_sent.append(words.numpy())
    words=paddle.unsqueeze(words,axis=-1)
    print(words)

res=np.stack(de_sent,axis=1)

for i in range(nums):
    print("-"*20)
    en_input=" ".join(datas[indices[i]][0])
    ground_truth_translate="".join(datas[indices[i]][1])
    
    pred=""
    for k in res[i]:
        w=list(cn_vocab)[k]
        if k<=2:
            continue
        pred+=w
    
    print(en_input)
    print("true:{}".format(ground_truth_translate))
    print("prde:{}".format(pred))


Tensor(shape=[10, 1], dtype=int64, place=Place(gpu:0), stop_gradient=True,
       [[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]])
Tensor(shape=[10, 3533], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[ 0.90092176, -5.54510689,  2.78532362, ..., -5.36725760,
         -4.54339314, -5.08433104],
        [ 0.45347211, -4.27472019,  1.85404038, ..., -4.41379166,
         -3.66277814, -4.44554901],
        [-1.88010931, -5.60863018,  2.97885060, ..., -4.89115047,
         -5.39066792, -5.19049501],
        ...,
        [-1.99162412, -5.58226919,  1.41175151, ..., -5.56767988,
         -5.42175579, -5.77841043],
        [-2.98571825, -8.45469189,  4.97252989, ..., -8.26989651,
         -7.82386971, -8.93665123],
        [-3.04714942, -6.81268501,  1.32199883, ..., -6.20924950,
         -6.79948759, -6.55603933]])
Tensor(shape=[10], dtype=int64, place=Place(gpu:0), stop_gradient=False,
       [28 , 2